In [30]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
os.chdir('/Users/KevinShao/Desktop/adsi_project')

In [4]:
df = pd.read_csv('data/raw/beer_reviews.csv')

In [5]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [7]:
df = df.dropna()

In [8]:
print(df.shape)

(1518478, 13)


In [9]:
df_cleaned = df.copy()

In [10]:
df_cleaned.drop(['brewery_id','review_time','review_profilename','beer_name','beer_beerid', 'beer_abv'], axis=1, inplace=True)

In [11]:
target_col = 'beer_style'

In [12]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

In [13]:
sc = StandardScaler()

In [14]:
cat_cols = ['brewery_name','beer_style']

In [15]:
num_cols = list(set(df_cleaned.columns) - (set(cat_cols) | set([target_col])))

In [16]:
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [17]:
oe = OrdinalEncoder(handle_unknown='ignore')
ohe = OneHotEncoder(sparse=False)

In [18]:
X_cat = oe.fit_transform(df_cleaned[cat_cols])

In [19]:
X_cat = pd.DataFrame(X_cat, columns = cat_cols)

In [20]:
df_cleaned.drop(cat_cols ,axis=1, inplace=True)

In [21]:
df_cleaned.reset_index(drop=True, inplace=True)
X_cat.reset_index(drop=True, inplace=True)

In [22]:
X = pd.concat([df_cleaned, X_cat], axis=1)

In [33]:
from src.data.sets import split_sets_random

In [35]:
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col='beer_style', test_ratio=0.2, to_numpy=True)

In [36]:
from src.models.pytorch import PytorchDataset
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [37]:
from src.models.null import NullModel

In [38]:
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [39]:
from src.models.performance import print_class_perf

In [40]:
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.07444192974099043
F1 Training: 0.010315310209270104


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from src.models.pytorch import PytorchMultiClass
from src.models.pytorch import get_device

In [42]:
model = PytorchMultiClass()
device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=256, bias=True)
  (layer_2): Linear(in_features=256, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [171]:
EPOCHS = 5
BATCH_SIZE = 16

In [172]:
criterion = nn.CrossEntropyLoss()

In [173]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [174]:
from src.models.pytorch import train_classification, test_classification

In [175]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.2476	|	Acc: 8.8%
	(valid)	|	Loss: 0.2436	|	Acc: 9.1%
Epoch: 1
	(train)	|	Loss: 0.2447	|	Acc: 9.3%
	(valid)	|	Loss: 0.2409	|	Acc: 10.1%
Epoch: 2
	(train)	|	Loss: 0.2442	|	Acc: 9.3%
	(valid)	|	Loss: 0.2431	|	Acc: 9.7%
Epoch: 3
	(train)	|	Loss: 0.2439	|	Acc: 9.4%
	(valid)	|	Loss: 0.2428	|	Acc: 9.5%
Epoch: 4
	(train)	|	Loss: 0.2436	|	Acc: 9.5%
	(valid)	|	Loss: 0.2411	|	Acc: 10.0%


In [176]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

print(f'Epoch: {epoch}')
print(f'\t(train)\t|\tLoss: {test_loss:.4f}\t|\tAcc: {test_acc * 100:.1f}%')

Epoch: 4
	(train)	|	Loss: 0.2411	|	Acc: 9.9%


In [177]:
torch.save(model, "models/beer_review.pt")

In [44]:
from sklearn.pipeline import Pipeline

In [45]:
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [46]:
cat_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder(handle_unknown='ignore'))
    ]
)

In [47]:
from sklearn.compose import ColumnTransformer

In [48]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_cols', cat_transformer, cat_cols)
    ]
)

In [49]:
nn_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('nn', baseline_model)
    ]
)

In [50]:
from joblib import dump 
dump(model,'models/nn_pipeline.joblib')

['models/nn_pipeline.joblib']